In [64]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import itertools
pd.set_option('display.max_colwidth', -1)

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_UFC_events'
url_request = requests.get(url).text
soup = BeautifulSoup(url_request, 'html.parser')

In [3]:
events_data = []
table = soup.find('table',{'id':'Past_events'})
table_rows = table.find_all('tr')
for row in table_rows[1:]:
    events_data.append([t.text.strip() for t in row.find_all('td')]) 
            
events_df = pd.DataFrame(events_data, columns=['Index', 'Event', 'Date', 'Venue', 'Location', 'Attendance','Ref'])

In [4]:
links = []
for row in table_rows[1:]:
#     links.append([t.get('href') for t in row.find_all('a')]) 
    links.append("https://en.wikipedia.org" + row.find('a').get('href'))

In [5]:
events_df.insert(2, "wikipage", links) 

In [20]:
print("Total events: " + str(len(events_df)))
events_occurred_df = events_df[events_df['Attendance']!="Cancelled"]
print("Events (excluding cancelled): " + str(len(events_occurred_df)))

Total events: 523
Events (excluding cancelled): 513


In [96]:
events_df.to_csv("list_of_UFC_events.csv", index=False)

In [21]:
# len(events_occurred_df[279:])
events_occurred_df = events_occurred_df.reset_index(drop=True)
events_occurred_df

,Index,Event,wikipage,Date,Venue,Location,Attendance,Ref
0,513,UFC Fight Night: Lee vs. Oliveira,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,"Mar 14, 2020",Ginásio Nilson Nelson,"Brasília, Brazil",0,[14]
1,512,UFC 248: Adesanya vs. Romero,https://en.wikipedia.org/wiki/UFC_248,"Mar 7, 2020",T-Mobile Arena,"Las Vegas, Nevada, U.S.","15,077",[15]
2,511,UFC Fight Night: Benavidez vs. Figueiredo,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,"Feb 29, 2020",Chartway Arena,"Norfolk, Virginia, U.S.","7,098",[16]
3,510,UFC Fight Night: Felder vs. Hooker,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,"Feb 23, 2020",Spark Arena,"Auckland, New Zealand","10,025",[17]
4,509,UFC Fight Night: Anderson vs. Błachowicz 2,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,"Feb 15, 2020",Santa Ana Star Center,"Rio Rancho, New Mexico, U.S.","6,449",[18]
...,...,...,...,...,...,...,...,...
508,005,UFC 5: The Return of the Beast,https://en.wikipedia.org/wiki/UFC_5,"Apr 7, 1995",Independence Arena,"Charlotte, North Carolina, U.S.","6,000",[463]
509,004,UFC 4: Revenge of the Warriors,https://en.wikipedia.org/wiki/UFC_4,"Dec 16, 1994",Expo Square Pavilion,"Tulsa, Oklahoma, U.S.","5,857",[464]
510,003,UFC 3: The American Dream,https://en.wikipedia.org/wiki/UFC_3,"Sep 9, 1994",Grady Cole Center,"Charlotte, North Carolina, U.S.",N/A,
511,002,UFC 2: No Way Out,https://en.wikipedia.org/wiki/UFC_2,"Mar 11, 1994",Mammoth Gardens,"Denver, Colorado, U.S.","2,000",[465]


In [8]:
result_df = pd.DataFrame()

In [23]:
for index, row in events_occurred_df.iterrows():
# for index, row in itertools.islice(events_occurred_df.iterrows(), 3):
    event_name = row['Event']
    event_url = row['wikipage']
    event_date = row['Date']
    print(str(index) + "\t" + event_date + "\t" + event_name)
    try:
        url_request = requests.get(event_url).text
        soup = BeautifulSoup(url_request, 'html.parser')
        data = []
        table = soup.find('table',{'class':'toccolours'})
        table_rows = table.find_all('tr')
        for row in table_rows:
            data.append([t.text.strip() for t in row.find_all('td')]) 
        df = pd.DataFrame(data, columns=['Weight class', 'Fighter1', 'Result', 'Fighter2', 'Method', 'Round', 'Time','Note'])
        df = df[~df['Weight class'].isnull()]
        df.insert(0, "Date", event_date) 
        df.insert(1, "Event", event_name)
    except:
        print("Error")
#     df.to_csv("data/"+ event_name + ".csv", index=False)
    result_df = result_df.append(df, ignore_index=True)

0	Mar 14, 2020	UFC Fight Night: Lee vs. Oliveira
1	Mar 7, 2020	UFC 248: Adesanya vs. Romero
2	Feb 29, 2020	UFC Fight Night: Benavidez vs. Figueiredo
3	Feb 23, 2020	UFC Fight Night: Felder vs. Hooker
4	Feb 15, 2020	UFC Fight Night: Anderson vs. Błachowicz 2
5	Feb 8, 2020	UFC 247: Jones vs. Reyes
6	Jan 25, 2020	UFC Fight Night: Blaydes vs. dos Santos
7	Jan 18, 2020	UFC 246: McGregor vs. Cowboy
8	Dec 21, 2019	UFC Fight Night: Edgar vs. The Korean Zombie
9	Dec 14, 2019	UFC 245: Usman vs. Covington
10	Dec 7, 2019	UFC on ESPN: Overeem vs. Rozenstruik
11	Nov 16, 2019	UFC Fight Night: Błachowicz vs. Jacaré
12	Nov 9, 2019	UFC Fight Night: Magomedsharipov vs. Kattar
13	Nov 2, 2019	UFC 244: Masvidal vs. Diaz
14	Oct 26, 2019	UFC Fight Night: Maia vs. Askren
15	Oct 18, 2019	UFC on ESPN: Reyes vs. Weidman
16	Oct 12, 2019	UFC Fight Night: Joanna vs. Waterson
17	Oct 5, 2019	UFC 243: Whittaker vs. Adesanya
18	Sep 28, 2019	UFC Fight Night: Hermansson vs. Cannonier
19	Sep 21, 2019	UFC Fight Night: Rodríg

162	Mar 5, 2016	UFC 196: McGregor vs. Diaz
163	Feb 27, 2016	UFC Fight Night: Silva vs. Bisping
164	Feb 21, 2016	UFC Fight Night: Cowboy vs. Cowboy
165	Feb 6, 2016	UFC Fight Night: Hendricks vs. Thompson
166	Jan 30, 2016	UFC on Fox: Johnson vs. Bader
167	Jan 17, 2016	UFC Fight Night: Dillashaw vs. Cruz
168	Jan 2, 2016	UFC 195: Lawler vs. Condit
169	Dec 19, 2015	UFC on Fox: dos Anjos vs. Cowboy 2
170	Dec 12, 2015	UFC 194: Aldo vs. McGregor
171	Dec 11, 2015	The Ultimate Fighter: Team McGregor vs. Team Faber Finale
172	Dec 10, 2015	UFC Fight Night: Namajunas vs. VanZant
173	Nov 28, 2015	UFC Fight Night: Henderson vs. Masvidal
174	Nov 21, 2015	The Ultimate Fighter Latin America 2 Finale: Magny vs. Gastelum
175	Nov 15, 2015	UFC 193: Rousey vs. Holm
176	Nov 7, 2015	UFC Fight Night: Belfort vs. Henderson 3
177	Oct 24, 2015	UFC Fight Night: Holohan vs. Smolka
178	Oct 3, 2015	UFC 192: Cormier vs. Gustafsson
179	Sep 27, 2015	UFC Fight Night: Barnett vs. Nelson
180	Sep 5, 2015	UFC 191: Johnson vs.

321	Dec 10, 2011	UFC 140: Jones vs. Machida
322	Dec 3, 2011	The Ultimate Fighter: Team Bisping vs. Team Miller Finale
323	Nov 19, 2011	UFC 139: Shogun vs. Henderson
324	Nov 12, 2011	UFC on Fox: Velasquez vs. dos Santos
325	Nov 5, 2011	UFC 138: Leben vs. Muñoz
326	Oct 29, 2011	UFC 137: Penn vs. Diaz
327	Oct 8, 2011	UFC 136: Edgar vs. Maynard III
328	Oct 1, 2011	UFC Live: Cruz vs. Johnson
329	Sep 24, 2011	UFC 135: Jones vs. Rampage
330	Sep 17, 2011	UFC Fight Night: Shields vs. Ellenberger
331	Aug 27, 2011	UFC 134: Silva vs. Okami
332	Aug 14, 2011	UFC Live: Hardy vs. Lytle
333	Aug 6, 2011	UFC 133: Evans vs. Ortiz
334	Jul 2, 2011	UFC 132: Cruz vs. Faber
335	Jun 26, 2011	UFC Live: Kongo vs. Barry
336	Jun 11, 2011	UFC 131: dos Santos vs. Carwin
337	Jun 4, 2011	The Ultimate Fighter: Team Lesnar vs. Team dos Santos Finale
338	May 28, 2011	UFC 130: Rampage vs. Hamill
339	Apr 30, 2011	UFC 129: St-Pierre vs. Shields
340	Mar 26, 2011	UFC Fight Night: Nogueira vs. Davis
341	Mar 19, 2011	UFC 128: Sh

503	May 17, 1996	UFC 9: Motor City Madness
504	Feb 16, 1996	UFC 8: David vs. Goliath
505	Dec 16, 1995	UFC: The Ultimate Ultimate
506	Sep 8, 1995	UFC 7: The Brawl in Buffalo
507	Jul 14, 1995	UFC 6: Clash of the Titans
508	Apr 7, 1995	UFC 5: The Return of the Beast
509	Dec 16, 1994	UFC 4: Revenge of the Warriors
510	Sep 9, 1994	UFC 3: The American Dream
511	Mar 11, 1994	UFC 2: No Way Out
512	Nov 12, 1993	UFC 1: The Beginning


In [24]:
result_df.to_csv("data/ufc_matches.csv", index=False)
result_df.to_json("data/ufc_matches.json",orient='records')

In [187]:
print(len(result_df))
print(len(result_df['Event'].unique()))

5707


504

In [132]:
# Need to inspect these pages
inspect_df = pd.DataFrame(np.array([['Apr 20, 2013', 'UFC on Fox: Henderson vs. Melendez','2'],
                                    ['Jan 19, 2013','UFC on FX: Belfort vs. Bisping','1'],
                                    ['Jul 21, 2012','UFC 149: Faber vs. Barão','5'],
                                    ['Jun 8, 2012','UFC on FX: Johnson vs. McCall','4'],
                                    ['May 15, 2012','UFC on Fuel TV: The Korean Zombie vs. Poirier','3'], 
                                    ['Feb 15, 2012','UFC on Fuel TV: Sanchez vs. Ellenberger','5'],
                                    ['Jan 28, 2012','UFC on Fox: Evans vs. Davis','4'], 
                                    ['Apr 5, 2007','UFC Fight Night: Stevenson vs. Guillard','1'], 
                                    ['Aug 6, 2005','UFC Ultimate Fight Night','-1']]), #no wiki page exists
                   columns=['date', 'event_name','table_number'])
supp_result_df = pd.DataFrame()

In [133]:
for index in range(0,8):
    try:
        event_date = inspect_df.iloc[index,0]
        event_name = inspect_df.iloc[index,1]
        table_number = int(inspect_df.iloc[index,2])
        event_url = events_occurred_df[events_occurred_df['Date']==event_date]['wikipage'].values[0]
        print(event_url)
        url_request = requests.get(event_url).text
        soup = BeautifulSoup(url_request, 'html.parser')
        data = []
        table = soup.findAll('table',{'class':'toccolours'})
        table_rows = table[table_number].find_all('tr')
        for row in table_rows:
            data.append([t.text.strip() for t in row.find_all('td')]) 
        df = pd.DataFrame(data, columns=['Weight class', 'Fighter1', 'Result', 'Fighter2', 'Method', 'Round', 'Time','Note'])
        df = df[~df['Weight class'].isnull()]
        df.insert(0, "Date", event_date) 
        df.insert(1, "Event", event_name)
        print(df.iloc[0,0:6])
    except:
        print("Error")
    supp_result_df = supp_result_df.append(df, ignore_index=True)

https://en.wikipedia.org/wiki/UFC_on_Fox:_Henderson_vs._Melendez
Date            Apr 20, 2013                      
Event           UFC on Fox: Henderson vs. Melendez
Weight class    Lightweight                       
Fighter1        Benson Henderson (c)              
Result          def.                              
Fighter2        Gilbert Melendez                  
Name: 2, dtype: object
https://en.wikipedia.org/wiki/UFC_on_FX:_Belfort_vs._Bisping
Date            Jan 19, 2013                  
Event           UFC on FX: Belfort vs. Bisping
Weight class    Middleweight                  
Fighter1        Vitor Belfort                 
Result          def.                          
Fighter2        Michael Bisping               
Name: 2, dtype: object
https://en.wikipedia.org/wiki/2012_in_UFC#UFC_149:_Faber_vs._Barão
Date            Jul 21, 2012            
Event           UFC 149: Faber vs. Barão
Weight class    Bantamweight            
Fighter1        Renan Barão             
Result   

In [189]:
print(len(supp_result_df))
print(len(supp_result_df['Event'].unique()))

90
8


In [190]:
# Find info on this event since no wiki page exists
event_date = 'Aug 6, 2005'
event_name = 'UFC Ultimate Fight Night'
# hhttps://www.ufc.com/event/UFC-Fight-Night-1
# https://www.sherdog.com/events/UFC-Fight-Night-1-Marquardt-vs-Salaverry-3100

In [191]:
data_array = np.array([
    ['Middleweight', 'Nate Marquardt', 'def.', 'Ivan Salaverry', 'Decision (unanimous) (30-27, 30-27, 29-28)', '3', '5:00', ''],
    ['Middleweight', 'Chris Leben', 'def.', 'Patrick Cote', 'Decision (split) (29-28, 30-27, 27-29)', '3', '5:00', ''],
    ['Light Heavyweight', 'Stephan Bonnar', 'def.', 'Sam Hoger', 'Decision (unanimous) (30-27, 30-27, 30-27)', '3', '5:00', ''],
    ['Middleweight', 'Nate Quarry', 'def.', 'Pete Sell', ' TKO (punch)', '1', '0:42', ''],
    ['Welterweight', 'Josh Koscheck', 'def.', 'Pete Spratt', 'Submission (rear-naked choke)', '1', '1:53', ''],
    ['Middleweight', 'Mike Swick', 'def.', 'Gideon Ray', 'TKO (punches)', '1', '0:22', ''],
    ['Welterweight', 'Kenny Florian', 'def.', 'Alex Karalexis', 'TKO (doctor stoppage)', '2', '2:52', ''],
    ['Welterweight', 'Drew Fickett', 'def.', 'Josh Neer', 'Submission (rear-naked choke)', '1', '1:35', '']])

In [194]:
df = pd.DataFrame(data_array,
                  columns=['Weight class', 'Fighter1', 'Result', 'Fighter2', 'Method', 'Round', 'Time','Note'])
df.insert(0, "Date", event_date) 
df.insert(1, "Event", event_name)
df

,Date,Event,Weight class,Fighter1,Result,Fighter2,Method,Round,Time,Note
0,"Aug 6, 2005",UFC Ultimate Fight Night,Middleweight,Nate Marquardt,def.,Ivan Salaverry,"Decision (unanimous) (30-27, 30-27, 29-28)",3,5:00,
1,"Aug 6, 2005",UFC Ultimate Fight Night,Middleweight,Chris Leben,def.,Patrick Cote,"Decision (split) (29-28, 30-27, 27-29)",3,5:00,
2,"Aug 6, 2005",UFC Ultimate Fight Night,Light Heavyweight,Stephan Bonnar,def.,Sam Hoger,"Decision (unanimous) (30-27, 30-27, 30-27)",3,5:00,
3,"Aug 6, 2005",UFC Ultimate Fight Night,Middleweight,Nate Quarry,def.,Pete Sell,TKO (punch),1,0:42,
4,"Aug 6, 2005",UFC Ultimate Fight Night,Welterweight,Josh Koscheck,def.,Pete Spratt,Submission (rear-naked choke),1,1:53,
5,"Aug 6, 2005",UFC Ultimate Fight Night,Middleweight,Mike Swick,def.,Gideon Ray,TKO (punches),1,0:22,
6,"Aug 6, 2005",UFC Ultimate Fight Night,Welterweight,Kenny Florian,def.,Alex Karalexis,TKO (doctor stoppage),2,2:52,
7,"Aug 6, 2005",UFC Ultimate Fight Night,Welterweight,Drew Fickett,def.,Josh Neer,Submission (rear-naked choke),1,1:35,


In [196]:
supp_result_df = supp_result_df.append(df)
print(len(supp_result_df))
print(len(supp_result_df['Event'].unique()))

98
9


In [197]:
result_df = result_df.append(supp_result_df)
print(len(result_df))
print(len(result_df['Event'].unique()))

5805
513


In [198]:
result_df.to_csv("data/ufc_matches.csv", index=False)
result_df.to_json("data/ufc_matches.json",orient='records')